<h1>Table of Contents<span class="tocSkip"></span></h1>


# Introduction

Our client is a real estate investment firm that would like  to break into the Texas housing market. The firm has asked us to identify the top 5 zip codes thye should invest in to ensure investment success.

# Data importation & cleaning

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from pandas import Grouper
from pandas import Series
from pandas import concat
import statsmodels.api as sm
from statsmodels.tsa.arima.model import  ARIMA
from statsmodels.tsa.statespace import sarimax
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMAResults
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import statsmodels.api as sm
import warnings
import pmdarima as pm


In [2]:
# import data
raw_df = pd.read_csv('data/zillow_data.csv')
df_income = pd.read_csv('data/zip_codes_2019_median_inc.csv')

## Cleaning column names

In [3]:
raw_df.columns = raw_df.columns.str.strip().str.lower()
df_income.columns = df_income.columns.str.strip().str.lower()

In [4]:
raw_df.rename(columns={'regionname': 'zipcode', 'countyname':'county'},inplace=True)
df_income.rename(columns={
    'zip code':'zipcode', 'population (2019)': 'population',
    'median family income (2019)': 'median_family_income'},
                 inplace=True)

## Stratifying to Texas

We begin to narrow down our dataset, first by filtering to only data from the state of Texas

In [5]:
raw_df = raw_df[raw_df['state']=='TX']
df_income['state'] = df_income['preferred name'].str.slice(-2)
df_income = df_income[df_income['state']=='TX']
df_income = (df_income[df_income['type'] != 'PO box'])

## Creating statistical columns

create ROI, Standard Deviation, Mean, and Coefficient of Variance statistical columns

In [6]:
#Calculate historical return on investment
raw_df['ROI']= (raw_df['2018-04']/raw_df['1996-04'])-1
raw_df['ROI']


#Calculate standard deviation of monthly values
raw_df['std']=raw_df.loc[:,'1996-04':'2018-04'].std(skipna=True, axis=1)

#Calculate historical mean value
raw_df['mean']=raw_df.loc[:,'1996-04':'2018-04'].mean(skipna=True, axis=1)

 
#Show calculated values
raw_df[['zipcode','std','mean','ROI']].head()

warnings.filterwarnings('ignore')

In [7]:
# RegionID is something from Zillow, don't need it and we don't need state any more.
raw_df.drop(['regionid', 'sizerank', 'metro'], axis=1,inplace=True)

In [8]:
# we only want zipcode, pop, mfi, lat and long from 2019 income data
df_income = df_income[df_income.columns[np.r_[0,5,7,9:11]]]
df_income

,zipcode,population,median_family_income,latitude,longitude
31142,73301,NaN,NaN,NaN,NaN
31143,73344,NaN,NaN,NaN,NaN
31744,75001,"14,992","$79,551",32.959999,-96.838997
31745,75002,"71,253","$105,656",33.090000,-96.609001
31746,75006,"51,642","$72,325",32.962002,-96.899002
...,...,...,...,...,...
34257,79961,NaN,NaN,NaN,NaN
34258,79968,NaN,NaN,NaN,NaN
34259,79976,NaN,NaN,NaN,NaN
34260,79978,NaN,NaN,NaN,NaN


In [9]:
# fixing mfi and pop columns
df_income['median_family_income'] = df_income['median_family_income'].str.strip('$\n\t')
df_income['median_family_income'] = df_income['median_family_income'].str.replace(',','')
df_income = df_income[df_income['median_family_income'].notna()]
df_income['median_family_income'] = df_income['median_family_income'].astype(int)
df_income['population'] = df_income['population'].str.replace(',','')
df_income['population'] = df_income['population'].astype(int)

## Melting our df

In [10]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['zipcode', 'city', 'state', 'county','ROI','std','mean'], var_name='date')
    melted['date'] = pd.to_datetime(melted['date'], infer_datetime_format=True)
    melted['year'] = [d.year for d in melted.date]
    melted = melted.dropna(subset=['value'])
    return melted

In [11]:
df = melt_data(raw_df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 255096 entries, 0 to 262084
Data columns (total 10 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   zipcode  255096 non-null  int64         
 1   city     255096 non-null  object        
 2   state    255096 non-null  object        
 3   county   255096 non-null  object        
 4   ROI      250160 non-null  float64       
 5   std      255096 non-null  float64       
 6   mean     255096 non-null  float64       
 7   date     255096 non-null  datetime64[ns]
 8   value    255096 non-null  float64       
 9   year     255096 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(2), object(3)
memory usage: 21.4+ MB


In [12]:
df.set_index(df['date'], inplace = True)
df.drop('date',axis=1, inplace=True)
df

,zipcode,city,state,county,ROI,std,mean,value,year
date,,,,,,,,,
1996-04-01,75070,McKinney,TX,Collin,0.365295,33537.101427,219655.849057,235700.0,1996
1996-04-01,77494,Katy,TX,Harris,0.567966,37730.794353,262110.566038,210400.0,1996
1996-04-01,79936,El Paso,TX,El Paso,0.571798,18167.079218,101875.471698,77300.0,1996
1996-04-01,77084,Houston,TX,Harris,0.729474,17242.109812,121185.283019,95000.0,1996
1996-04-01,77449,Katy,TX,Harris,0.846960,18603.288493,126530.188679,95400.0,1996
...,...,...,...,...,...,...,...,...,...
2018-04-01,76941,Mertzon,TX,Irion,NaN,13734.029878,99318.367347,121600.0,2018
2018-04-01,79313,Anton,TX,Hockley,NaN,5286.674724,54424.675325,63300.0,2018
2018-04-01,79355,Plains,TX,Yoakum,NaN,12929.465872,78967.346939,93500.0,2018


# Merging our data

In [13]:
df_merged = df.reset_index().merge(df_income, how='left').set_index('date')
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 255096 entries, 1996-04-01 to 2018-04-01
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   zipcode               255096 non-null  int64  
 1   city                  255096 non-null  object 
 2   state                 255096 non-null  object 
 3   county                255096 non-null  object 
 4   ROI                   250160 non-null  float64
 5   std                   255096 non-null  float64
 6   mean                  255096 non-null  float64
 7   value                 255096 non-null  float64
 8   year                  255096 non-null  int64  
 9   population            252181 non-null  float64
 10  median_family_income  252181 non-null  float64
 11  latitude              252181 non-null  float64
 12  longitude             252181 non-null  float64
dtypes: float64(8), int64(2), object(3)
memory usage: 27.2+ MB


In [14]:
# creating a separate df_merged to save this data for later
df_merged_for_later = df_merged

## Filtering on population

In [15]:
df_upper = df_merged['population'].quantile(.75)
df_filtered_pop = df_merged[(df_merged['population'] > df_upper)]

## Filtering on house value percentile

In [16]:
# Select the first quantile
q1 = df_filtered_pop['value'].quantile(.25)

# Select the third quantile
q3 = df_filtered_pop['value'].quantile(.75)


# Create a filtered in between q1 & q3
filtered = df_filtered_pop[(df_filtered_pop['value']>=q1) 
             & (df_filtered_pop['value']<=q3)]
df_filtered_iqr = pd.DataFrame(filtered)
df_filtered_iqr.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 31421 entries, 1996-04-01 to 2018-04-01
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   zipcode               31421 non-null  int64  
 1   city                  31421 non-null  object 
 2   state                 31421 non-null  object 
 3   county                31421 non-null  object 
 4   ROI                   31421 non-null  float64
 5   std                   31421 non-null  float64
 6   mean                  31421 non-null  float64
 7   value                 31421 non-null  float64
 8   year                  31421 non-null  int64  
 9   population            31421 non-null  float64
 10  median_family_income  31421 non-null  float64
 11  latitude              31421 non-null  float64
 12  longitude             31421 non-null  float64
dtypes: float64(8), int64(2), object(3)
memory usage: 3.4+ MB


In [17]:
# group by date and aggregate on median value
df_filtered_iqr['median_house_price'] = df_filtered_iqr.groupby('date').aggregate({'value':'median'})
df_filtered_iqr['affordability'] = df_filtered_iqr['value'] - df_filtered_iqr['median_family_income']/.12

## Filtering on affordability score

Affordability Score is a metric to represent the ability for a resident to pay their mortgage.

It is based off two assumptions:

    1.Your yearly mortgage should be ~30% of your annual salary
    2.It's a 30 year mortgage

In [18]:
# create function to normalize data
def normalize_data(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))
df_filtered_iqr['affordability_score']  = normalize_data(df_filtered_iqr['affordability'])

In [19]:
# filtering to the 20th percentile
top_20 = df_filtered_iqr['affordability_score'].quantile(.5)
df_filtered_afs = df_filtered_iqr[(df_filtered_iqr['affordability_score'] < top_20)]
df_filtered_afs.reset_index().isna().sum()

date                    0
zipcode                 0
city                    0
state                   0
county                  0
ROI                     0
std                     0
mean                    0
value                   0
year                    0
population              0
median_family_income    0
latitude                0
longitude               0
median_house_price      0
affordability           0
affordability_score     0
dtype: int64

## Filtering for 2017-2018

In [20]:
df_filtered_2017 = df_filtered_afs['2017-01-01':].dropna()

In [21]:
print(df_filtered_top_roi['population'].median())
print(df_filtered_top_roi['value'].median())

NameError: name 'df_filtered_top_roi' is not defined

### Sort by highest ROI

In [ ]:
df_filtered_2017 = df_filtered_2017.reset_index()
df_filtered_top_roi = df_filtered_2017.loc[df_filtered_2017.groupby("zipcode")["ROI"].idxmax()].sort_values('ROI',axis=0,ascending=False)[:5]
df_filtered_top_roi.set_index('date',inplace=True)
df_filtered_top_roi

# EDA on Best Zipcodes

## Pulling out best zips

In [ ]:
zipcodes = list(df_filtered_top_roi['zipcode'].values)
zipcodes

In [ ]:
df_filtered_top_roi = df_filtered_top_roi[df_filtered_top_roi['zipcode'].isin(zipcodes)]

`df_zip_5` is filtered for the top 5 zip codes.

Each unique row represents a combination of unique month and zip code.

In [ ]:
df_zip_5 = df_merged[df_merged['zipcode'].isin(zipcodes)]

In [ ]:
df_zip_5['zipcode'].value_counts()

## Unstack zips

`df_prices` will unstack zips through a pivot function.

Its values represent the home prices per month.

In [ ]:
df_prices = pd.DataFrame(df_zip_5[['zipcode','value']], index = df_zip_5.index)
df_prices.reset_index(inplace=True)

df_prices = df_prices.pivot('date', 'zipcode', 'value')
df_prices

# Modeling on Each Zipcode

In each model, we split the zipcode data so we train on prices from April 1996 - April 2016 and test on prices from May 2016 - April 2018

## Defining Stationarity, TS Metrics, and Plot Functions

In [ ]:
def stationarity_check(df):     
    df_test = adfuller(df)
    print('Results of Dickey-Fuller Test: \n')

    dfoutput = pd.Series(df_test[0:4], index=['Test Statistic', 'p-value', 
                                             '#Lags Used', 'Number of Observations Used'])
    for key,value in df_test[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    return None

In [ ]:
def arimamodel(timeseries):
    automodel = pm.auto_arima(timeseries, 
                              start_p=1, 
                              start_q=1,
                              test="adf",
                              seasonal=False,
                              trace=True)
    return automodel

In [ ]:
def plotarima(n_periods, timeseries, automodel):
    # Forecast
    fc, confint = automodel.predict(n_periods=n_periods, 
                                    return_conf_int=True)
    # Weekly index
    fc_ind = pd.date_range(timeseries.index[timeseries.shape[0]-1], 
                           periods=n_periods, 
                           freq="W")
    # Forecast series
    fc_series = pd.Series(fc, index=fc_ind)
    # Upper and lower confidence bounds
#     lower_series = pd.Series(confint[:, 0], index=fc_ind)
#     upper_series = pd.Series(confint[:, 1], index=fc_ind)
    # Create plot
    plt.figure(figsize=(10, 6))
    plt.plot(timeseries)
    plt.plot(fc_series, color="red")
    plt.xlabel("date")
    plt.ylabel(timeseries.name)
#     plt.fill_between(lower_series.index, 
#                      lower_series, 
#                      upper_series, 
#                      color="k", 
#                      alpha=0.25)
    plt.legend(("past", "forecast"))
    plt.show()



In [ ]:
def arima_results(endog, test, order=[0,0,0]):
    endog= train.astype('float32')  
    test = test.astype('float32')  
    arima = ARIMA(endog, order=order).fit()
    
    train_hat = arima.predict()
    
    rmse_train = mean_squared_error(endog,
                                    train_hat,
                                    squared=False)
    
    
    rmse_test = mean_squared_error(test,
                                   arima.predict(test.index[0], test.index[-1]), 
                                   squared=False)
    
 
    print(f"""Train RMSE: {rmse_train}""")     
    print(f"""Test RMSE: {rmse_test}""")    
    print(f"""Summary: {arima.summary()}""")
    
    return sm

In [ ]:
def sarimax_results(endog, test, order=[0,0,0], seasonal_order=[0,0,0,0]):
      
    sm = SARIMAX(endog, order=order, seasonal_order=seasonal_order).fit()
    
    rmse_train = mean_squared_error(endog,
                                    sm.predict(train.index[0], train.index[-1], typ='levels'),
                                    squared=False)
    
    
    rmse_test = mean_squared_error(test,
                                   sm.predict(test.index[0], test.index[-1], typ='levels'), 
                                   squared=False)
    
    
    print(f""" Train RMSE: {rmse_train}""")     
    print(f""" Test RMSE: {rmse_test}""") 
    print(f""" Summary: {sm.summary()}""")
    return sm



In [ ]:
def plot_preds_sarimax(test, sarimax=sarimax):
    sarimax.predict(test.index[0], test.index[-1], typ='levels').plot()
    test.plot()

## 75052

In [ ]:
df_75052 = pd.DataFrame(df_prices[75052][df_prices[75052].notnull()]).astype(int) 
df_75052['value'] = df_75052[75052] 
df_75052.drop(df_75052.columns[np.r_[1]], axis=1, inplace=True)

In [ ]:
df_75052

In [ ]:
train = df_75052[:-24]
test = df_75052[241:]
train.plot();

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = sm.tsa.seasonal_decompose(train, model='additive')
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
# Plot gathered statistics
plt.figure(figsize=(14,15))
plt.subplot(411)
plt.plot(train, label='Original', color='blue')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend', color='blue')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality', color='blue')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals', color='blue')
plt.legend(loc='best')
plt.tight_layout()



In [ ]:
fig, ax = plt.subplots()
ax.set_title('Yearly Home Prices', fontsize=15)
ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('Home Prices', fontsize=14)
sns.lineplot(data=train, linewidth=4)
fig.set_size_inches(15, 8);
plt.savefig('images/avg_aff_score')


In [ ]:
print(f"""Train date range: {train.idxmin()} to {train.idxmax()}\n\n
Test date range: {test.idxmin()} to {test.idxmax()}
""")

In [ ]:
roll_mean = train.rolling(window=12).mean()
roll_mean.plot();

In [ ]:
# Apply a 1st order difference to the time series and plot the rolling mean

rm_diff_1 = roll_mean.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

**Stationarity Check** 
> We can see that after adding rolling_mean and differencing twice, the Dickey Fuller test improved. We will test out an ARIMA model and SARIMAX to see which performs better.  

In [ ]:
print(stationarity_check(df_75052))
print(stationarity_check(rm_diff_2.dropna()))

**Determine best ARIMA hyper paraters**

In [ ]:
timeseries = df_75052[75052]
automodel = arimamodel(df_prices[75052])
plotarima(70, timeseries, automodel)

**ARIMA Model Results**

In [ ]:
model = arima_results(endog = train, test = test, order=[4,2,2])

**SARIMAX Model Train and Test Results**

In [ ]:
# set print_ts_metrics equal to model, then feed that model to plot_predictions
model = sarimax_results(endog=train, test=test, order=[1,1,0], seasonal_order=[1,1,0,12])
plot_preds_sarimax(test, model);

## 75104

In [ ]:
df_75104 = pd.DataFrame(df_prices[75104][df_prices[75104].notnull()]).astype(int) 
df_75104['value'] = df_75104[75104] 
df_75104.drop(df_75104.columns[np.r_[1]], axis=1, inplace=True)

In [ ]:
train = df_75104[:-24]
test = df_75104[241:]
train.plot();

In [ ]:
roll_mean = train.rolling(window=12).mean()
roll_mean.plot();

In [ ]:
rm_diff_1 = roll_mean.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

**Stationarity Check** 
> We can see that after adding rolling_mean and differencing twice, the Dickey Fuller test improved. We will test out an ARIMA model and SARIMAX to see which performs better. 

In [ ]:
print(stationarity_check(df_75104))
print(stationarity_check(rm_diff_2.dropna()))

**Determine best ARIMA hyper paraters**

In [ ]:
timeseries = df_75104[75104]
automodel = arimamodel(df_prices[75104])
plotarima(70, timeseries, automodel)

**ARIMA Model Results**

In [ ]:
model_train = arima_results(train,test,  order=[1,2,2])


**SARIMAX Model Results**

In [ ]:
model = sarimax_results(endog=train, test=test, order=[4,2,2], seasonal_order=[1,1,0,12])
plot_preds_sarimax(test, model);

## 76108

In [ ]:
df_76108 = pd.DataFrame(df_prices[76108][df_prices[76108].notnull()]).astype(int) 
df_76108['value'] = df_76108[76108] 
df_76108.drop(df_76108.columns[np.r_[1]], axis=1, inplace=True)

In [ ]:
train = df_76108[:-24]
test = df_76108[241:]
train.plot();

In [ ]:
roll_mean = train.rolling(window=12).mean()
roll_mean.plot();

In [ ]:
rm_diff_1 = roll_mean.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))


rm_diff_2 = roll_mean.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

**Stationarity Check** 
> We can see that after adding rolling_mean and differencing twice, the Dickey Fuller test improved. We will test out an ARIMA model and SARIMAX to see which performs better. 

In [ ]:
print(stationarity_check(df_76108))
print(stationarity_check(rm_diff_2.dropna()))

**Determine ARIMA best hyper parameters**

In [ ]:
timeseries = df_76108[76108] 
automodel = arimamodel(df_prices[76108])
plotarima(70, timeseries, automodel)

**ARIMA Model Results**

In [ ]:
model_train = arima_results(train, test,  order=[2,2,0])

**SARIMAX Model Train and Test Results**

In [ ]:
model = sarimax_results(endog=train, test=test, order=[1,1,1], seasonal_order=[1,1,0,12])
plot_preds_sarimax(test, model);

## 77571

In [ ]:
df_77571 = pd.DataFrame(df_prices[77571][df_prices[77571].notnull()]).astype(int) 
df_77571['value'] = df_77571[77571] 
df_77571.drop(df_77571.columns[np.r_[1]], axis=1, inplace=True)

In [ ]:
train = df_77571[:-24]
test = df_77571[241:]
train.plot();

In [ ]:
roll_mean = train.ewm(halflife=4).mean()
roll_mean.plot();

In [ ]:
rm_diff_1 = roll_mean.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))

In [ ]:
rm_diff_2 = roll_mean.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

**Stationarity Check** 
> We can see that after adding rolling_mean and differencing twice, the Dickey Fuller test improved. We will test out an ARIMA model and SARIMAX to see which performs better. 

In [ ]:
print(stationarity_check(df_77571))
print(stationarity_check(rm_diff_2.dropna()))

**Determine ARIMA best hyper parameters**

In [ ]:
timeseries = df_77571[77571]
automodel = arimamodel(df_prices[77571])
plotarima(70, timeseries, automodel)

**ARIMA Model Results**

In [ ]:
model_train = arima_results(train, test, order=[2,1,1])

**SARIMAX Model Train and Test Results**

In [ ]:
model = sarimax_results(endog=train, test=test, order=[1,1,0], seasonal_order=[1,1,0,12])
plot_preds_sarimax(test, model);

## 79423

In [ ]:
df_79423 = pd.DataFrame(df_prices[79423][df_prices[79423].notnull()]).astype(int) 
df_79423['value'] = df_79423[79423] 
df_79423.drop(df_79423.columns[np.r_[1]], axis=1, inplace=True)


In [ ]:
train = df_79423[:-24]
test = df_79423[241:]
train.plot();

In [ ]:
roll_mean = train.rolling(window=12).mean()
roll_mean.plot();

In [ ]:
rm_diff_1 = roll_mean.diff(periods=1)
rm_diff_1.plot(figsize=(12,6))

In [ ]:
rm_diff_2 = roll_mean.diff(periods=1).diff(periods=1)
rm_diff_2.plot(figsize=(12,6));

**Stationarity Check** 
> We can see that after adding rolling_mean and differencing twice, the Dickey Fuller test improved. We will test out an ARIMA model and SARIMAX to see which performs better. 

In [ ]:
print(stationarity_check(df_79423))
print(stationarity_check(rm_diff_2.dropna()))

**Determine ARIMA best hyper parameters**

In [ ]:
timeseries = df_79423[79423]
automodel = arimamodel(df_prices[79423])
plotarima(70, timeseries, automodel)

**ARIMA Model Results**

In [ ]:
model_train = arima_results(train,test,  order=[1,1,0])

**SARIMAX Model Results**

In [ ]:
model = sarimax_results(endog=train, test=test, order=[1,1,0], seasonal_order=[1,1,0,12])
plot_preds_sarimax(test, model);

# Visualizations

In [ ]:
df_viz = pd.DataFrame(df_merged.copy())
df_viz['affordability'] = df_viz['value'] - df_viz['median_family_income']/.12
df_viz['affordability_score']  = normalize_data(df_viz['affordability'])
df_viz_5 = df_viz[df_viz['zipcode'].isin(zipcodes)]
df_viz_5

**Average Affordability Score**

In [ ]:
df_afs_tx = df_viz.groupby('date').aggregate({'affordability_score':'median'})
df_afs_tx

In [ ]:
df_afs_5 = pd.DataFrame(df_viz_5[['zipcode','affordability_score']], index=df_viz_5.index)
df_afs_5.reset_index(inplace=True)
df_afs_5 = df_afs_5.pivot('date', 'zipcode', 'affordability_score')
df_afs_5

In [ ]:
df_afs = df_afs_5.merge(df_afs_tx, left_index=True, right_index=True)
df_afs.rename(columns={'affordability_score': 'Texas'}, inplace=True)

In [ ]:
df_afs = df_afs[['Texas', 75052, 75104, 76108, 77571, 79423]]
df_afs

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Average Affordability Score', fontsize=15)
ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('Score', fontsize=14)
sns.lineplot(data=df_afs, linewidth=4)
fig.set_size_inches(15, 8);
plt.savefig('images/avg_aff_score')


**Average ROI by Zipcode**

In [ ]:
df_roi_5 = pd.DataFrame(df_viz_5[['zipcode','ROI']], index=df_viz_5.index)
df_roi_5.reset_index(inplace=True)
df_roi_5 = df_roi_5.pivot('date', 'zipcode', 'ROI')


In [ ]:
fig, ax = plt.subplots()
ax.set_title('Average ROI', fontsize=15)
ax.set_xlabel('Zip', fontsize=14)
ax.set_ylabel('ROI', fontsize=14)
sns.barplot(data=df_roi_5, palette= "hls", 
            linewidth=4,order=[75104, 75052,76108,77571,79423]).axhline(df_filtered_iqr['ROI'].mean(),linewidth=5,color='black')
fig.set_size_inches(15, 8)
plt.savefig('images/average_roi')

**Median Home Price**

In [ ]:
df_iqr = df_zip_5.groupby('date').aggregate({'value':'median'})

In [ ]:
fig, ax = plt.subplots()
ax.set_title('Yearly Home Prices', fontsize=15)
ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('Median $', fontsize=14)
sns.lineplot(data=df_iqr, linewidth=4)
fig.set_size_inches(15, 8);
plt.savefig('images/avg_aff_score')

In [ ]:
# # import folium
# import folium

# # this is about the center of Texas per google search
# lat = 31.5
# long = -99.9

# # create base texas map
# tx_map = folium.Map(location=[lat, long],min_lat=df_income['latitude'].min(),max_lat=df_income['latitude'].max(),
#                       min_lon=df_income['longitude'].min(),max_lon=df_income['longitude'].max(),
#                       max_bounds=True,zoom_start=6)

# # plotting our points
# x = df_income['latitude']
# y = df_income['longitude']
# points = list(zip(x, y))
# for p in points:
#     lat = p[0]
#     long = p[1]
#     marker = folium.Circle(radius=5,location=[lat, long],color='green')
#     marker.add_to(tx_map)
    
# # plotting our main zips
# x5 = df_filtered_top_roi['latitude']
# y5 = df_filtered_top_roi['longitude']
# c5 = df_filtered_top_roi['city']
# z5 = df_filtered_top_roi['zipcode']
# main_points = list(zip(x5,y5,c5,z5))
# for p in main_points:
#     lat = p[0]
#     long = p[1]
#     city = p[2]
#     zips = p[3]
#     popup_text = "Zip: {} \n\n City: {}".format(zips, city)
#     popup = folium.Popup(popup_text,parse_html=True)
#     marker = folium.Marker(location=[lat,long], popup=popup)
#     marker.add_to(tx_map)
    
# tx_map